***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-369f  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import math

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Dec 28 10:26 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'bucket316164417' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.endswith('.parquet'):
        paths.append(full_path+b.name)
        
bucket_name = 'projectdata316164417'

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [8]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [9]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [10]:
from inverted_index_gcp import InvertedIndex

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [11]:
parquetFile = spark.read.parquet(*paths)

In [12]:
# Count number of wiki pages
#parquetFile.count()

In [13]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(id, text):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  cnt = Counter()
  for token in tokens:
    if token not in all_stopwords:
      cnt[token] += 1
  return [(token,(id, val)) for token, val in cnt.items()]

def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl)

def reduce_dict(unsorted_pl):
  return sorted(set(unsorted_pl))

def calculate_df(postings):
  return postings.map(lambda x: (x[0],len(x[1])))

def partition_postings_and_write(postings, base_dir):
  return postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey().map(lambda x: InvertedIndex.write_a_posting_list(x, base_dir, bucket_name))

def partition_postings_and_write_anchor(postings, base_dir):
  return postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey().map(lambda x: InvertedIndex.write_a_posting_list_anchor(x, base_dir, bucket_name))

In [14]:
def tokens_each_doc(text):
  return [token.group() for token in RE_WORD.finditer(text.lower()) if token not in all_stopwords]

def DL_nfi(posting):
  totalLength = 0
  totalPow2 = 0
  for term, tf in posting:
    totalLength += tf
    totalPow2 += math.pow(tf, 2)
  return (totalLength, 1 / math.sqrt(totalPow2))

def anchor_tokenizing(lst):
  tokens = []
  for item in lst:
    tokens.extend([(token.group(), item[0]) for token in RE_WORD.finditer(item[1].lower()) if token not in all_stopwords])
  return tokens

**TITLE:**


In [15]:
doc_pairs_title = parquetFile.select("id", "title").rdd
doc_to_title = doc_pairs_title.collectAsMap()
DL_title = doc_pairs_title.map(lambda x: (x[0], len(tokens_each_doc(x[1])))).collectAsMap()
word_counts_title = doc_pairs_title.flatMap(lambda x: word_count(x[0], x[1]))
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)
# postings_filtered_title = postings_title.filter(lambda x: len(x[1])>50)
w2df_title = calculate_df(postings_title)
w2df_dict_title = w2df_title.collectAsMap()
posting_locs_list_title = partition_postings_and_write(postings_title, 'title_index').collect()

In [16]:
# collect all posting lists locations into one super-set
super_posting_locs_title = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title_index'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_title[k].extend(v)

# Create inverted index instance
inverted_title = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_title.posting_locs = super_posting_locs_title
# Add the token - df dictionary to the inverted index
inverted_title.df = w2df_dict_title
inverted_title.DL = DL_title
inverted_title.N = len(DL_title)
total_val = 0.0
for x in DL_title.values():
  total_val += x
inverted_title.avgdl = total_val / inverted_title.N
# write the global stats out
inverted_title.write_index('.', 'title')
# upload to gs
index_src = "title.pkl"
index_dst = f'gs://{bucket_name}/title_index/{index_src}'
!gsutil cp $index_src $index_dst

In [17]:
with open(Path('.') / f'doc_to_title.pkl', 'wb') as f:
    pickle.dump(doc_to_title, f)
src = "doc_to_title.pkl"
dst = f'gs://{bucket_name}/{src}'
!gsutil cp $src $dst

**BODY:**

In [18]:
doc_pairs_text = parquetFile.select("id", "text").rdd

In [19]:
# DL_text = doc_pairs_text.map(lambda x: (x[0], DL_nfi(x[1]))).collectAsMap()

In [20]:
word_counts_text = doc_pairs_text.flatMap(lambda x: word_count(x[0], x[1]))

In [21]:
postings_text = word_counts_text.groupByKey().mapValues(reduce_word_counts)
postings_filtered_text = postings_text.filter(lambda x: len(x[1])>50)

In [ ]:
DL_text = postings_filtered_text.flatMap(lambda x: [(tup[0], (x[0], tup[1])) for tup in x[1]]).groupByKey().mapValues(list).map(lambda x: (x[0], DL_nfi(x[1]))).collectAsMap()

In [22]:
w2df_text = calculate_df(postings_filtered_text)

In [ ]:
w2df_dict_text = w2df_text.collectAsMap()

In [ ]:
posting_locs_list_text = partition_postings_and_write(postings_filtered_text, 'body_index').collect()

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs_text = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='body_index'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_text[k].extend(v)

# Create inverted index instance
inverted_text = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_text.posting_locs = super_posting_locs_text
# Add the token - df dictionary to the inverted index
inverted_text.df = w2df_dict_text
inverted_text.DL = DL_text
inverted_text.N = len(DL_text)
total_val = 0.0
for x in DL_text.values():
  total_val += x[0]
inverted_text.avgdl = total_val / inverted_text.N
# write the global stats out
inverted_text.write_index('.', 'body')
# upload to gs
index_src = "body.pkl"
index_dst = f'gs://{bucket_name}/body_index/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://body.pkl [Content-Type=application/octet-stream]...
| [1 files][129.9 MiB/129.9 MiB]                                                
Operation completed over 1 objects/129.9 MiB.                                    


**ANCHOR:**


In [27]:
doc_pairs_anchor = parquetFile.select("id", "anchor_text").rdd
word_counts_anchor = doc_pairs_anchor.flatMap(lambda x: anchor_tokenizing(x[1]))
postings_anchor = word_counts_anchor.groupByKey().mapValues(reduce_dict)
# postings_filtered_anchor = postings_anchor.filter(lambda x: len(x[1])>50)
w2df_anchor = calculate_df(postings_anchor)
w2df_dict_anchor = w2df_anchor.collectAsMap()
posting_locs_list_anchor = partition_postings_and_write_anchor(postings_anchor, 'anchor_index').collect()

In [28]:
# collect all posting lists locations into one super-set
super_posting_locs_anchor = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='anchor_index'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_anchor[k].extend(v)

# Create inverted index instance
inverted_anchor = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_anchor.posting_locs = super_posting_locs_anchor
# Add the token - df dictionary to the inverted index
inverted_anchor.df = w2df_dict_anchor
# write the global stats out
inverted_anchor.write_index('.', 'anchor')
# upload to gs
index_src = "anchor.pkl"
index_dst = f'gs://{bucket_name}/anchor_index/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://anchor.pkl [Content-Type=application/octet-stream]...
\ [1 files][ 95.0 MiB/ 95.0 MiB]                                                
Operation completed over 1 objects/95.0 MiB.                                     


In [29]:
!gsutil ls -lh $index_dst

 95.02 MiB  2022-12-28T15:30:08Z  gs://bucket316164417/anchor_index/anchor.pkl
TOTAL: 1 objects, 99632460 bytes (95.02 MiB)


# PageRank

In [30]:
# Put your `generate_graph` function here
def generate_graph(pages):
  edges = pages.flatMap(lambda x: set([(x[0], y[0]) for y in x[1]]))
  vertices = pages.flatMap(lambda x: [(x[0], )] + [(y[0],) for y in x[1]]).distinct()
  return edges, vertices

In [ ]:
# pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(doc_pairs_anchor)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('id'))
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr.show()

+---+------------------+
| id|          pagerank|
+---+------------------+
| 12| 147.7334534423781|
| 25| 59.46853992954965|
| 39| 57.71570853205964|
|290|31.598711370141082|
|303|361.74497456349076|
|305| 33.85560357229939|
|307| 330.1845085435861|
|308|387.29475534179966|
|309|5.7470679397511635|
|316| 38.02341498210656|
|324| 512.0423004762725|
|330|  0.26680664641919|
|332| 4.111818467779909|
|334|21.560735631698087|
|336| 31.28590516741786|
|339| 36.05089151175037|
|340| 3.351263223337941|
|344| 6.885160497491854|
|358|484.09636397351534|
|359|0.6129794446734336|
+---+------------------+
only showing top 20 rows



In [ ]:
with open(Path('.') / f'pagerank.pkl', 'wb') as f:
    pickle.dump(pr.toPandas().set_index('id')['pagerank'].to_dict(), f)
src = "pagerank.pkl"
dst = f'gs://{bucket_name}/{src}'
!gsutil cp $src $dst

Copying file://pagerank.pkl [Content-Type=application/octet-stream]...
\ [1 files][ 84.7 MiB/ 84.7 MiB]                                                
Operation completed over 1 objects/84.7 MiB.                                     


In [ ]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})

--2022-12-28 16:05:52--  https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2503235912 (2.3G) [application/octet-stream]
Saving to: ‘pageviews-202108-user.bz2’

pageviews-202108-us 100%[===================>]   2.33G  4.67MB/s    in 8m 45s  

2022-12-28 16:14:37 (4.55 MB/s) - ‘pageviews-202108-user.bz2’ saved [2503235912/2503235912]



In [ ]:
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
  pickle.dump(wid2pv, f)
src = pv_clean
dst = f'gs://{bucket_name}/{src}'
!gsutil cp $src $dst

Copying file://pageviews-202108-user.pkl [Content-Type=application/octet-stream]...
- [1 files][ 73.5 MiB/ 73.5 MiB]                                                
Operation completed over 1 objects/73.5 MiB.                                     
